<a href="https://colab.research.google.com/github/Ads369/Ads_2s/blob/main/ipynb/Lesson%2025/25_4_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Навигация по уроку**

1. [Первое знакомство с AutoML](https://colab.research.google.com/drive/1bCWyzlp1-tcvt7TE60m4hFnRd5AWscWY)
2. [Гиперпараметры и оптимизация моделей](https://colab.research.google.com/drive/1CN69NftfVXUliyv11FGbM7qOYbO0XON5)
3. [AutoML в Keras](https://colab.research.google.com/drive/1V7mfY8da0S-FbWxhQbchJM38JSJBmtoZ)
4. Домашняя работа

В домашней работе необходимо с помощью AutoKeras или KerasTuner найти оптимальную модель для решения одной из следующей задач:

1. На 3 балла. Обучите модель с точностью не менее 90% предсказывать сарказм в новостных заголовках. Составьте 5 произвольных заголовков, которых нет в датасете и проверьте на них обученную модель, сделайте выводы. Ссылка на [датасет](https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip).
2. На 4 балла. Используйте [русский корпус новостей от Lenta.ru](https://www.kaggle.com/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/data) подберите и обучите модель классифицировать новости по заголовкам на классы (поле topic в датасете). Используйте 9 самых часто встречаемых топиков и 10-й для остальных, не вошедших в 9 классов. Оцените модель с помощью отчета о классификации, сделайте выводы.  
3. На 5 баллов. Найдите публичный датасет по обращениям граждан в администрацию, техническую поддержку или за консультацией. Обучите модель классифицировать обращения по тематикам. Сформируйте отчет о классификации и матрицу ошибок.

In [2]:
!wget https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip
!unzip -qo "Sarcasm_Headlines_Dataset_v2.json.zip" -d ./dataset

--2025-01-09 14:45:47--  https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1789636 (1.7M) [application/x-zip-compressed]
Saving to: ‘Sarcasm_Headlines_Dataset_v2.json.zip.3’

Sarcasm_Headlines_D 100%[===================>]   1.71M  1.88MB/s    in 0.9s    

2025-01-09 14:45:48 (1.88 MB/s) - ‘Sarcasm_Headlines_Dataset_v2.json.zip.3’ saved [1789636/1789636]



In [3]:
!pip install tensorflow==2.15.1

In [4]:
!pip install autokeras==1.1.0 tensorflow==2.15.1 keras-nlp==0.5.1

In [5]:
import tensorflow as tf
tf.__version__

'2.15.1'

In [6]:
import keras_nlp as nlp
nlp.__version__

'0.5.1'

In [7]:
# Библиотека матричного вычисления
import numpy as np
# Библиотека для работы с данными
import pandas as pd
# Библиотека для работы с регулярными выражениями
import re
# Библиотека для работы с фреймворком TensorFlow
import tensorflow as tf
# Библиотека AutoML autokeras
import autokeras as ak
# Библиотеки для построения графиков и их стилизации
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Утилита для расщепления выборки
from sklearn.model_selection import train_test_split

# Необходимые метрики для построения Матрицы ошибок и отчета о классификации
from sklearn.metrics import classification_report, confusion_matrix

In [8]:
address = "./dataset/Sarcasm_Headlines_Dataset_v2.json"
json_df = pd.read_json(address, lines = True ) # библиотека pandas умеет работать с json данными
df_sarcasm = pd.DataFrame(json_df) # создаем датафрейм

df_sarcasm.head() # выводим первые 5 записей датафрейма

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [9]:
# Ссылки на сами статьи нам не нужны
df_sarcasm = df_sarcasm.drop("article_link", axis = 1)

In [10]:
print('Найдено дубликатов: ', df_sarcasm.duplicated().sum())

# Удаляем дубликаты
df_sarcasm.drop_duplicates(subset=['headline'], inplace = True)

print('Осталось дубликатов после очистки: ', df_sarcasm.duplicated().sum())

Найдено дубликатов:  116
Осталось дубликатов после очистки:  0


In [11]:
X_train, X_tmp, y_train, y_tmp = train_test_split(np.array(df_sarcasm.headline),
                                                  np.array(df_sarcasm.is_sarcastic),
                                                  test_size=0.3)

X_val, X_test, y_val, y_test = train_test_split(X_tmp,
                                                  y_tmp,
                                                  test_size=0.5)


In [12]:
print('Форма входных данных: ', X_train.shape)
print('Форма выходных меток: ', y_train.shape)
print('Пример заголовка: ', X_train[0])

Форма входных данных:  (19952,)
Форма выходных меток:  (19952,)
Пример заголовка:  'syrians' lives are worthless,' obama tells daughters before kissing them goodnight


In [13]:
# Для экономии ОЗУ удаляем уже ненужные данные, после чего python запускает сборщик "мусора" для очистки памяти
del df_sarcasm, X_tmp, y_tmp

In [ ]:
# Создание ансамбля моделей (3 наиболее подходящие к данным моделей)
clf = ak.TextClassifier(overwrite=True, max_trials=3, objective='val_accuracy')

# Обучаем 2 модели на 4-х эпохах, размер пакета подбирается автоматически
result_training = clf.fit(X_train, y_train, epochs=6, validation_data=(X_val, y_val))



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
vanilla           |vanilla           |text_block_1/block_type
0                 |0                 |classification_head_1/dropout
5000              |5000              |text_block_1/max_tokens
False             |False             |text_block_1/conv_block_1/separable
512               |512               |text_block_1/text_to_int_sequence_1/output_sequence_length
none              |none              |text_block_1/embedding_1/pretraining
64                |64                |text_block_1/embedding_1/embedding_dim
0.25              |0.25              |text_block_1/embedding_1/dropout
5                 |5                 |text_block_1/conv_block_1/kernel_size
1                 |1                 |text_block_1/conv_block_1/num_blocks
1                 |1                 |text_block_1/conv_block_1/num_layers
False             |False             |text_block_1/conv_block_1/max_pooling
0                 |0            